In [1]:
from pathlib import Path
from pprint import pprint

import pandas as pd
import numpy as np

from bayes_opt import BayesianOptimization

from isic_helper import get_folds
from isic_helper import compute_pauc, compute_auc

In [2]:
id_column = "isic_id"
target_column = "target"
ensemble_column = "oof_preds_ensemble"

model_names = ["cb", "lgb", "efficientnet_b0", "efficientnet_b1"]
versions = ["v1", "v3", "v2", "v1"]
paths = [f"/kaggle/input/isic-scd-{model_name.replace('_', '-')}-{version}-train" for model_name, version in zip(model_names, versions)]

In [3]:
val_auc_scores = {}
val_pauc_scores = {}
for idx, path in enumerate(paths):
    model_name = model_names[idx]
    version = versions[idx]
    oof_preds_model_df = pd.read_csv(f"{path}/oof_preds_{model_name}_{version}.csv")
    if idx == 0:
        oof_preds_df = oof_preds_model_df.copy()
    else:
        oof_preds_df = oof_preds_df.merge(oof_preds_model_df[[id_column, f"oof_{model_name}_{version}"]],
                                          on="isic_id", how="inner")
        assert oof_preds_df.shape[0] == oof_preds_model_df.shape[0]
    val_auc_scores[f"{model_name}_{version}"] = {}
    val_pauc_scores[f"{model_name}_{version}"] = {}
val_auc_scores["ensemble"] = {}
val_pauc_scores["ensemble"] = {}

In [4]:
oof_columns = [col for col in oof_preds_df.columns if col.startswith("oof")]
all_folds = np.unique(oof_preds_df["fold"])

In [5]:
def blend_optimizer(oof_preds_df, oof_columns, init_points = 20, n_iter = 100):
    pbounds = {f"w{i}": (0.0, 1.0) for i in range(len(oof_columns))}

    def dim_opt(oof_preds_df, *args):
        weights = args
        score = 0
        for fold in all_folds:
            fold_ensemble_preds = 0
            for weight, oof_column in zip(weights, oof_columns):
                fold_ensemble_preds += weight * oof_preds_df.loc[oof_preds_df["fold"] == fold, oof_column].rank(pct=True).values
            score += compute_pauc(np.array(oof_preds_df.loc[oof_preds_df["fold"] == fold, "target"]), fold_ensemble_preds)
        return score / len(all_folds)
    
#     def dim_opt(oof_preds_df, *args):
#         weights = args
#         score = 0
#         for fold in all_folds:
#             oof_fold_preds_df = oof_preds_df[oof_preds_df["fold"] == fold]
#             fold_ensemble_preds = np.zeros((oof_fold_preds_df.shape[0], len(weights)))
#             for idx, (weight, oof_column) in enumerate(zip(weights, oof_columns)):
#                 fold_ensemble_preds[:, idx] = weight * np.log1p(oof_fold_preds_df[oof_column].rank(pct=True).values)
#             fold_ensemble_preds = np.exp(fold_ensemble_preds.sum(axis=1) / sum(weights)) - 1
#             score += compute_pauc(oof_fold_preds_df["target"], fold_ensemble_preds)
#         return score / len(all_folds)
    
#     def dim_opt(oof_preds_df, *args):
#         weights = args
#         score = 0
#         for fold in all_folds:
#             oof_fold_preds_df = oof_preds_df[oof_preds_df["fold"] == fold]
#             fold_ensemble_preds = np.zeros((oof_fold_preds_df.shape[0], len(weights)))
#             for idx, (weight, oof_column) in enumerate(zip(weights, oof_columns)):
#                 fold_ensemble_preds[:, idx] = weight * oof_fold_preds_df[oof_column].rank(pct=True).values
#             fold_ensemble_preds = fold_ensemble_preds.sum(axis=1) ** (1 / sum(weights))
#             score += compute_pauc(oof_fold_preds_df["target"], fold_ensemble_preds)
#         return score / len(all_folds)

    def q(**ws):
        ws = tuple(ws.values())
        return dim_opt(oof_preds_df, *ws)

    optimizer = BayesianOptimization(
        f=q,
        pbounds=pbounds,
        random_state=2022,
    )

    optimizer.maximize(
        init_points=init_points,
        n_iter=n_iter,
    )
    
    weights = []
    for i in range(len(oof_columns)):
        weights.append(optimizer.max["params"][f"w{i}"])
    
    print(f"Best pAUC: {optimizer.max['target']}")
    print(f"Best weights: {weights}")
    return weights


weights = blend_optimizer(
    oof_preds_df, oof_columns, 
    init_points=40, 
    n_iter=40
)

|   iter    |  target   |    w0     |    w1     |    w2     |    w3     |
-------------------------------------------------------------------------
| 1         | 0.1739    | 0.009359  | 0.4991    | 0.1134    | 0.04997   |
| 2         | 0.177     | 0.6854    | 0.487     | 0.8977    | 0.6475    |
| 3         | 0.1773    | 0.897     | 0.7211    | 0.8314    | 0.8276    |
| 4         | 0.1764    | 0.8336    | 0.957     | 0.368     | 0.4948    |
| 5         | 0.1755    | 0.3395    | 0.6194    | 0.9775    | 0.09643   |
| 6         | 0.1774    | 0.7442    | 0.2925    | 0.2987    | 0.7525    |
| 7         | 0.1751    | 0.01866   | 0.5237    | 0.8644    | 0.3888    |
| 8         | 0.1769    | 0.2122    | 0.4752    | 0.5647    | 0.3494    |
| 9         | 0.1759    | 0.9759    | 0.03782   | 0.7943    | 0.3579    |
| 10        | 0.1775    | 0.748     | 0.9145    | 0.3727    | 0.9649    |
| 11        | 0.1737    | 0.08139   | 0.04245   | 0.2968    | 0.3637    |
| 12        | 0.1773    | 0.4903    | 

In [6]:
weights

[0.4954245304424095,
 0.5202731555301114,
 0.3651203172131123,
 0.7922660075802592]

In [7]:
all_folds = np.unique(oof_preds_df["fold"])
for fold in all_folds:
    fold_index = oof_preds_df[oof_preds_df["fold"] == fold].index
    fold_target = oof_preds_df.loc[fold_index, target_column]
    fold_ensemble_preds = 0
    for model_name, version, weight in zip(model_names, versions, weights):
        fold_model_preds = oof_preds_df.loc[fold_index, f"oof_{model_name}_{version}"]
        fold_ensemble_preds += fold_model_preds.rank(pct=True).values * weight 
        
        val_auc_scores[f"{model_name}_{version}"][f"fold_{fold}"] = compute_auc(fold_target, fold_model_preds)
        val_pauc_scores[f"{model_name}_{version}"][f"fold_{fold}"] = compute_pauc(fold_target, fold_model_preds, min_tpr=0.8)
    
    oof_preds_df.loc[fold_index, ensemble_column] = fold_ensemble_preds
    val_auc_scores["ensemble"][f"fold_{fold}"] = compute_auc(fold_target, fold_ensemble_preds)
    val_pauc_scores["ensemble"][f"fold_{fold}"] = compute_pauc(fold_target, fold_ensemble_preds, min_tpr=0.8)

for model_name, version, weight in zip(model_names, versions, weights):
    print(f"Model: {model_name}_{version} | Weightage: {weight}")
    
    print("Val AUC scores:")
    pprint(val_auc_scores[f"{model_name}_{version}"])
    print("Val PAUC scores:")
    pprint(val_pauc_scores[f"{model_name}_{version}"])
    
    cv_model_auc_oof = compute_auc(oof_preds_df[target_column], oof_preds_df[f"oof_{model_name}_{version}"])
    cv_model_pauc_oof = compute_pauc(oof_preds_df[target_column], oof_preds_df[f"oof_{model_name}_{version}"], min_tpr=0.8)

    cv_model_auc_avg = np.mean(list(val_auc_scores[f"{model_name}_{version}"].values()))
    cv_model_pauc_avg = np.mean(list(val_pauc_scores[f"{model_name}_{version}"].values()))

    cv_model_auc_std = np.std(list(val_auc_scores[f"{model_name}_{version}"].values()))
    cv_model_pauc_std = np.std(list(val_pauc_scores[f"{model_name}_{version}"].values()))
    
    print(f"CV AUC OOF: {cv_model_auc_oof}")
    print(f"CV PAUC OOF: {cv_model_pauc_oof}")
    print(f"CV AUC AVG: {cv_model_auc_avg}")
    print(f"CV PAUC AVG: {cv_model_pauc_avg}")
    print(f"CV AUC STD: {cv_model_auc_std}")
    print(f"CV PAUC STD: {cv_model_pauc_std}")
    print("\n")

print("Val AUC scores:")
pprint(val_auc_scores["ensemble"])
print("Val PAUC scores:")
pprint(val_pauc_scores["ensemble"])

cv_ensemble_auc_oof = compute_auc(oof_preds_df[target_column], oof_preds_df[ensemble_column])
cv_ensemble_pauc_oof = compute_pauc(oof_preds_df[target_column], oof_preds_df[ensemble_column], min_tpr=0.8)

cv_ensemble_auc_avg = np.mean(list(val_auc_scores["ensemble"].values()))
cv_ensemble_pauc_avg = np.mean(list(val_pauc_scores["ensemble"].values()))

cv_ensemble_auc_std = np.std(list(val_auc_scores["ensemble"].values()))
cv_ensemble_pauc_std = np.std(list(val_pauc_scores["ensemble"].values()))

print(f"CV AUC OOF: {cv_ensemble_auc_oof}")
print(f"CV PAUC OOF: {cv_ensemble_pauc_oof}")
print(f"CV AUC AVG: {cv_ensemble_auc_avg}")
print(f"CV PAUC AVG: {cv_ensemble_pauc_avg}")
print(f"CV AUC STD: {cv_ensemble_auc_std}")
print(f"CV PAUC STD: {cv_ensemble_pauc_std}")

Model: cb_v1 | Weightage: 0.4954245304424095
Val AUC scores:
{'fold_1': 0.9523881910669536,
 'fold_2': 0.9535923162036922,
 'fold_3': 0.9457296489498446,
 'fold_4': 0.9721251762738515,
 'fold_5': 0.9470801267252167}
Val PAUC scores:
{'fold_1': 0.1645271757829003,
 'fold_2': 0.1669476383467947,
 'fold_3': 0.15652509640463735,
 'fold_4': 0.17601833248056303,
 'fold_5': 0.15814944833701147}
CV AUC OOF: 0.9524292434775488
CV PAUC OOF: 0.16241203053404624
CV AUC AVG: 0.9541830918439118
CV PAUC AVG: 0.16443353827038137
CV AUC STD: 0.009459406837513047
CV PAUC STD: 0.006965201549198192


Model: lgb_v3 | Weightage: 0.5202731555301114
Val AUC scores:
{'fold_1': 0.9576815612354752,
 'fold_2': 0.9508534742767404,
 'fold_3': 0.9506974516728857,
 'fold_4': 0.9630832636557638,
 'fold_5': 0.9616494948942257}
Val PAUC scores:
{'fold_1': 0.16882108869081178,
 'fold_2': 0.16124885048313303,
 'fold_3': 0.16251325953750728,
 'fold_4': 0.1684323170807802,
 'fold_5': 0.17159456378130328}
CV AUC OOF: 0.95469

In [8]:
oof_preds_df

,isic_id,patient_id,fold,target,oof_cb_v1,oof_lgb_v3,oof_efficientnet_b0_v2,oof_efficientnet_b1_v1,oof_preds_ensemble
0,ISIC_0015670,IP_1235828,4,0,0.000104,0.000015,0.000229,0.000009,0.926491
1,ISIC_0015845,IP_8170065,1,0,0.456523,0.652166,0.000568,0.001830,2.079563
2,ISIC_0015864,IP_6724798,5,0,0.000034,0.000019,0.000008,0.000001,0.342428
3,ISIC_0015902,IP_4111386,2,0,0.000022,0.000024,0.000538,0.000165,1.262225
4,ISIC_0024200,IP_8313778,1,0,0.000522,0.000160,0.000778,0.000059,1.721048
...,...,...,...,...,...,...,...,...,...
401054,ISIC_9999937,IP_1140263,3,0,0.003126,0.002748,0.001353,0.002319,2.035578
401055,ISIC_9999951,IP_5678181,3,0,0.000053,0.000030,0.000007,0.000018,0.659106
401056,ISIC_9999960,IP_0076153,2,0,0.001993,0.000537,0.000140,0.000022,1.456453
401057,ISIC_9999964,IP_5231513,5,0,0.000071,0.000028,0.000209,0.000271,1.222779


In [9]:
oof_preds_df.pivot_table(index="target", values=oof_columns+["oof_preds_ensemble"], aggfunc="mean")

,oof_cb_v1,oof_efficientnet_b0_v2,oof_efficientnet_b1_v1,oof_lgb_v3,oof_preds_ensemble
target,,,,,
0,0.000671,0.000992,0.000815,0.000313,1.085588
1,0.034607,0.035246,0.051048,0.021116,2.073024


In [10]:
oof_preds_df[oof_columns+["oof_preds_ensemble"]].corr()

,oof_cb_v1,oof_lgb_v3,oof_efficientnet_b0_v2,oof_efficientnet_b1_v1,oof_preds_ensemble
oof_cb_v1,1.000000,0.757476,0.224245,0.268655,0.187973
oof_lgb_v3,0.757476,1.000000,0.182886,0.208978,0.123773
oof_efficientnet_b0_v2,0.224245,0.182886,1.000000,0.539082,0.219788
oof_efficientnet_b1_v1,0.268655,0.208978,0.539082,1.000000,0.197820
oof_preds_ensemble,0.187973,0.123773,0.219788,0.197820,1.000000
